In [86]:
import os

In [87]:
API_KEY_METEOSOURCE = os.environ['API_KEY_METEOSOURCE']
API_KEY_NEWSAPI = os.environ['API_KEY_NEWSAPI']
API_KEY_OPENAI = os.environ['API_KEY_OPENAI']

In [88]:
import requests

In [89]:
newsapi_url_parameters = {
    'apiKey': API_KEY_NEWSAPI,
    'country': 'US'
}

In [90]:
import requests

response = requests.get('https://newsapi.org/v2/top-headlines', 
            params=newsapi_url_parameters) 
#response status_code
if response.status_code == 200:
    response_data = response.json()
    headlines_articles = [
        {'title': article['title'], 'description': article['description']}
        for article in response_data['articles']
    ]
    print(headlines_articles)
    for article in response_data['articles']:
        if article['title'] != '[Removed]':
            print(article['title'])
else: 
    print(response.text)
    raise SystemExit('Something went wrong!')

[{'title': 'NFL Week 18 Team Motivation - NBC Sports', 'description': "The Lions and Vikings are set to battle for the NFC's No. 1 seed while the Bengals and Dolphins need some help to make the playoffs heading into Week 18."}, {'title': 'South Korea plane crash: Why was there a wall near the runway? - BBC.com', 'description': 'An air safety expert says lives could have been saved if the "obstruction" was not there.'}, {'title': '[Removed]', 'description': '[Removed]'}, {'title': 'Liam Payne Death Probe Leads to Charges Against Five People, Including Alleged Friend - Hollywood Reporter', 'description': 'The One Direction singer died after falling from the balcony of his Buenos Aires hotel room in October.'}, {'title': 'Federal appeals court upholds sexual abuse verdict against Trump - CNN', 'description': 'A federal appeals court upheld the jury’s verdict finding Donald Trump sexually abused writer E. Jean Carroll and denied his request for a new trial.'}, {'title': 'Trump endorses Joh

**Weather API setup**

In [91]:
meteosource_headers = {
    'X-API-Key': API_KEY_METEOSOURCE
}

In [92]:
meteosource_findplaces_url_parameters = {
    'text': 'Berlin'
}

In [93]:
response = requests.get('https://www.meteosource.com/api/v1/free/find_places', 
                        params=meteosource_findplaces_url_parameters,
                        headers = meteosource_headers)

In [94]:
if response.status_code == 200:
    response_data = response.json()
    place_id = response_data[0]['place_id']
    print(place_id)
    
else:
    print(f"Error: {response.status_code} - {response.text}")  # Updated to print the error details
    raise SystemExit('Something went wrong, please inspect the response')

berlin


In [95]:
#create a dictionary for the url parameters
meteosource_point_url_parameters = {
    'place_id': place_id,
    'sections': 'daily'
}

In [96]:
response = requests.get('https://www.meteosource.com/api/v1/free/point',
                        headers = meteosource_headers,
                        params=meteosource_point_url_parameters
                        
                       )

In [97]:
if response.status_code == 200:
    response_data = response.json()
    weather_forecast = response_data['daily']['data'][0]['summary']
    print(weather_forecast)
    
else:
    print(f"Error: {response.status_code} - {response.text}")  # Updated to print the error details
    raise SystemExit('Something went wrong, please inspect the response')

Cloudy changing to sunny by afternoon. Temperature 0/3 °C.


**Create System Message variable**

In [98]:
system_message = ''' 
You are an AI assistant tasked with generating a 'Morning Update' text that’s engaging and enjoyable for the user to listen to while having their morning coffee. The update should be about 2-5 minutes long, incorporating both a weather forecast and top 10 news headlines in a way that feels conversational, lively, and fits a specific tone (such as funny, serious, sarcastic, or motivational). Do not output anything else than the text, don't include any markup, lists, or other structural elements. The text will be sent to a text-to-speech API to generate an MP3, so make sure the output contains nothing that should not be read out loud.

Structure the monologue as follows:

1. Greeting: Start with a warm and welcoming greeting.
2. Weather Summary: Describe the day’s weather, infusing the chosen tone (e.g., funny, serious, etc.) to make it engaging.
3. News Headlines: Present each headline in the chosen tone, followed by a summary of the headline to give the listener a deeper insight into the headline.
4. Closing: Wrap up with a concluding remark that leaves the reader with a smile, positive thought, or playful nudge.

Be creative in how you incorporate the tone and style, ensuring that the text is engaging and enjoyable to listen to.
'''

**Creating the user_message variable**

In [99]:
user_message = f'''
Please generate a 'Morning Update' text in a funny and light tone.

Here is the Weather Forecast: 
{weather_forecast}

Here are the News Headlines in JSON format:
{headlines_articles}

Generate the text as specified in the system prompt, following the structure of greeting, weather summary, 10 headlines, and a closing remark.
'''

In [100]:
openai_headers = {
    'Authorization': f'Bearer {API_KEY_OPENAI}'
}

In [101]:
completions_request_data = {
    'model': 'gpt-4o-mini',
    'messages': [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
}

In [102]:
import requests

response = requests.post('https://api.openai.com/v1/chat/completions',
                         headers=openai_headers,
                         json=completions_request_data
                         )

if response.status_code == 200:
    response_json = response.json()
    morning_update = response_json['choices'][0]['message']['content']
    print(morning_update)
elif response.status_code == 429:
    raise SystemExit('Error: Insufficient quota. Please check your plan and billing details.')
else:
    raise SystemExit(f'Something went wrong, please check response text: {response.text}')

Good morning, sunshine! Grab your coffee, get comfy, and let’s dive into this delightful morning update—perfectly brewed just for you!

Now, let’s take a peek outside, shall we? Today, we’re starting off under a blanket of clouds that seem to have forgotten how to part ways, but fear not! By this afternoon, the sunshine will break through, just like that one friend who always shows up fashionably late to brunch. The temperature will be a brisk 0 to 3 degrees Celsius, so remember, if you leave the house without a proper jacket, you might just turn into a popsicle!

Now, let’s sizzle up some news morsels. In sports, the NFL is heating up as the Lions and Vikings prepare to duke it out for the NFC’s very own golden ticket—the No. 1 seed for the playoffs! Talk about motivation! Meanwhile, the Bengals and Dolphins are hoping for just a sprinkle of luck to help them slide into the playoff party. 

Over in South Korea, a plane crash investigation is raising eyebrows and questions about some m

In [103]:
import requests

tts_request_data = {
    'model': 'tts-1',
    'input': morning_update, 
    'voice': 'fable'
}

response = requests.post('https://api.openai.com/v1/audio/speech',
                         headers=openai_headers,
                         json=tts_request_data)

if response.status_code == 200:
    file = open('morning_update.mp3', 'wb')
    file.write(response.content)
    file.close()
else:
    raise SystemExit('Something went wrong')